In [24]:
import numpy as np
import cv2 
import mediapipe as mp
import tensorflow as tf
import keras.models
gpu=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0],True)


In [25]:
mp_draw=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
# model = keras.models.load_model('weight_pose.h5')

In [26]:
def detection(img,model):
    # img=cv2.flip(img,1)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img.flags.writeable=False
    results=model.process(img)
    img.flags.writeable=True
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return img,results


In [27]:
def draw_edit(landmarks,frame):
    h,w,_=frame.shape
    nose=np.array([[landmarks[mp_pose.PoseLandmark.NOSE].x*w,landmarks[mp_pose.PoseLandmark.NOSE].y*h]])
    lft_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y*h]])
    rght_shoulder=np.array([[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x*w,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y*h]])
    neck=((lft_shoulder+rght_shoulder)/2+nose)/2
    chest_mid=(lft_shoulder+rght_shoulder)/2
    x1,y1=np.squeeze(neck)
    x2,y2=np.squeeze(chest_mid)
    cv2.circle(frame, (int(x1), int(y1)), 6, (0,255,0), -1)
    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,255,255), 4)
    

In [28]:
def lay_mau(landmark):
    a=[]
    lst_x=[]
    lst_y=[]
    for idx,lmk in enumerate(landmark):
        if idx in [x for x in range(1,11)]:
            continue
        elif idx in [x for x in range(17,23)]:
            continue
        # cv2.putText(frame,'Cor:{}'.format(y),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1,cv2.LINE_AA)
        if idx in [0,12,11,23,24,27,28]:
            lst_x.append(lmk.x)
            lst_y.append(lmk.y)
        a.append(lmk.x)
        a.append(lmk.y)
        a.append(lmk.z)
    return a,lst_x,lst_y

In [29]:
def xulylandmark(lst_x,lst_y,frame):
    waistx=(lst_x[3]+lst_x[4])/2*frame.shape[1]
    waisty=(lst_y[3]+lst_y[4])/2*frame.shape[0]
    xcenter=(lst_x[0]*frame.shape[1]+waistx)/2
    ycenter=(lst_y[0]*frame.shape[0]+waisty)/2
    footx=(lst_x[-2]+lst_x[-1])/2*frame.shape[1]
    widh=abs(xcenter-footx)
    footy=(lst_y[-2]+lst_y[-1])/2*frame.shape[0]
    heit=abs(ycenter-footy)
    return widh,heit
    

In [31]:
lst_sample=[]
frame_num=30
label=0
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5,model_complexity=1) as pose:
    cap=cv2.VideoCapture('C:/Users/PhuTuan/Downloads/TEST.mp4')
    # cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret,frame=cap.read()
        if ret:
            # frame=cv2.flip(frame,1)
            img,results=detection(frame,pose)
            if label==1:
                mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                mp_draw.DrawingSpec(color=(0,100,255),thickness=2,circle_radius=2),
                mp_draw.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=2))
                cv2.putText(img,'Fall',(50,50),
                cv2.FONT_HERSHEY_COMPLEX,2,(205,50,255),2,cv2.LINE_AA)
            else:
                mp_draw.draw_landmarks(img,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                mp_draw.DrawingSpec(color=(0,100,255),thickness=2,circle_radius=2),
                mp_draw.DrawingSpec(color=(255,255,255),thickness=2,circle_radius=2))
                cv2.putText(img,'Not Fall',(50,50),
                cv2.FONT_HERSHEY_COMPLEX,2,(205,50,255),2,cv2.LINE_AA)
            try:
                landmark=results.pose_landmarks.landmark
                lst_sample,lst_x,lst_y=lay_mau(landmark)
                # print(lst_x,lst_y)
                width,height=xulylandmark(lst_x,lst_y,img)
                if width>height:
                    label=1
                else:
                    label=0
                # if len(lst_sample)==30:
                #     tensor=np.expand_dims(lst_sample,axis=0)
                #     lst_sample=[]
                #     # action=model.predict(tensor)
                #     if action[0,0]>=0.7:
                #         label=1
                #     else:
                #         label=0
            except:
                pass
            # cv2.putText(img,str(label),(50,50),cv2.FONT_HERSHEY_COMPLEX,2,(205,50,255),2,cv2.LINE_AA)
            if cv2.waitKey(20)&0xFF==ord('q'):
                break
            # img=cv2.flip(img,1)
            cv2.imshow('Pose',img)
        else:
            break
    cap.release()
cv2.destroyAllWindows()